In [1]:
country_tickers =  ["EWW",  # iShares MSCI Mexico Inv. Mt. Idx
                    "ERUS", # iShares MSCI Russia ETF
                    "EWQ",  # iShares MSCI France Index ETF
                    "EWY",  # iShares MSCI South Korea Index ETF
                    "EWZ",  # iShares MSCI Brazil Index ETF
                    "EPOL", # ishares MSCI Poland ETF
                    "THD",  # iShares MSCI Thailand ETF
                    "NGE",  # GlobalX MSCI Nigeria ETF
                    "ARGT", # GlobalX MSCI Argentina ETF
                    "EPHE", # iShares MSCI Philippines ETF
                    "EWA",  # iShares MSCI Australia ETF
                    "ECH",  # iShares MSCI Chile Investable Market Index ETF
                   "MCHI", # iShares MSCI China ETF
                   "EGPT", #VanEck Egypt Index ETF
                   "INDA", #iShares MSCI India ETF
                   "EIDO", #iShares MSCI Indonesia ETF
                   "EPU", #iShares MSCI Peru ETF
                   "EZA", #iShares MSCI South Africa ETF
                   "EWT", #iShares MSCI Taiwan ETF
                   "TUR", #iShares MSCI Turkey ETF
                   ]
country_developed = ["EWJ",  # iShares MSCI Japan Index ETF
                    "EFNL", # iShares MSCI Finland Market Index ETF
                    "IVV",  # iShares S&P 500 Index
                    "EWH",  # iShares MSCI Hong Kong Index ETF
                    "EWI",  # iShares MSCI Italy Index ETF
                    "EWP",  # iShares MSCI Spain Index ETF
                    "EWD",  # iShares MSCI Sweden Index ETF
                    "EWL",  # iShares MSCI Switzerland Index ETF
                    "EWC",  # iShares MSCI Canada Index ETF
                    "EWO",  # iShares MSCI Austria Investable Mkt Index ETF
                    "EWK"  # iShares MSCI Belgium Investable Market Index ETF
                    ]


In [7]:
# Import libraries
import pandas as pd;
import numpy as np;

import requests;
import json;

from tqdm import tqdm;

supported_tickers = pd.read_csv(r"C:/Users/duongluong/Desktop/finpros/tiingo/tiingo_supported_tickers.csv")
supported_tickers = supported_tickers[supported_tickers.assetType == "ETF"];
supported_tickers = supported_tickers.dropna(0);
supported_tickers = supported_tickers[supported_tickers.ticker.isin(country_tickers)].reset_index(drop=True);
supported_tickers = supported_tickers[supported_tickers.endDate == "2023-03-23"]
print(supported_tickers)

   ticker   exchange assetType priceCurrency   startDate     endDate
0    ARGT  NYSE ARCA       ETF           USD  2011-03-03  2023-03-23
1     ECH       BATS       ETF           USD  2007-11-20  2023-03-23
2    EFNL       BATS       ETF           USD  2012-01-26  2023-03-23
3    EPHE  NYSE ARCA       ETF           USD  2010-09-29  2023-03-23
4    EPOL  NYSE ARCA       ETF           USD  2010-05-26  2023-03-23
6     EWA  NYSE ARCA       ETF           USD  1996-04-01  2023-03-23
7     EWC  NYSE ARCA       ETF           USD  1996-04-01  2023-03-23
8     EWD  NYSE ARCA       ETF           USD  1996-04-01  2023-03-23
9     EWH  NYSE ARCA       ETF           USD  1996-04-01  2023-03-23
10    EWI  NYSE ARCA       ETF           USD  1996-04-01  2023-03-23
11    EWJ  NYSE ARCA       ETF           USD  1996-04-01  2023-03-23
12    EWK  NYSE ARCA       ETF           USD  1996-04-01  2023-03-23
13    EWL  NYSE ARCA       ETF           USD  1996-04-01  2023-03-23
14    EWO  NYSE ARCA       ETF    

C:\Users\duongluong\AppData\Local\Temp\ipykernel_10024\4153789684.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  supported_tickers = supported_tickers.dropna(0);


In [10]:
tiingo_token = '09c8cf45aa0af105efa9f4160a5efc9def76140b'
headers = {
'Content-Type': 'application/json',
'Authorization' : f'Token {tiingo_token}'
}
select_columns = ["date", "adjClose", "adjHigh", "adjLow", "adjOpen", "adjVolume"];
rename_columns = ["Timestamp", "Close", "High", "Low", "Open", "Volume"];
# Scrape Data From Tiingo

for ticker, start_date, end_date in tqdm(supported_tickers[["ticker", "startDate", "endDate"]].values):
    try:
        url = f"https://api.tiingo.com/tiingo/daily/{ticker}/prices?startDate={start_date}&endDate={end_date}&format=json&resampleFreq=daily";

        r = requests.get(url, headers=headers).json()

        df = pd.DataFrame(r)
        df = df[select_columns];
        df.columns = rename_columns;
        df.Timestamp = pd.to_datetime(df.Timestamp)

        df.to_parquet(f"C:/Users/duongluong/Desktop/finpros/tiingo/data/daily_{ticker}_ohlcv_tiingo.parquet")
    except Exception as e:
        print(f"Error Occured With {ticker}: ", e)

100%|██████████| 22/22 [03:06<00:00,  8.50s/it]


In [ ]:
import pandas as pd
from datetime import time,date,datetime,timedelta
import requests
import numpy as np
import trade_package

def get_etf_data():
    etf_file_names = trade_package.get_file_name('C:/Users/duongluong/Desktop/finpros/tiingo/data')
    etf_data = pd.DataFrame()
    for etf_file_str in etf_file_names:
        etf_temp = pd.read_parquet(f"C:/Users/duongluong/Desktop/finpros/tiingo/data/{etf_file_str}")
        etf_name = etf_file_str.split("_")[1]
        etf_temp = etf_temp.set_index(["Timestamp"], drop=True)
        etf_temp.rename(columns= lambda col: col +" "+ etf_name, inplace=True)
        etf_data = pd.concat([etf_data, etf_temp], axis=1)
    return etf_data

etf_df = pd.read_csv(r'C:/Users/duongluong/Desktop/finpros/tiingo/etf_data.csv')
print(etf_df)
def format_date_pd(series):
    #series in the form : pandas date range
    yymmdd = list(map(lambda x: int(x), str(series).split(" ")[0].split("-")))
    return datetime(int(yymmdd[0]),int(yymmdd[1]),int(yymmdd[2]))
etf_df["Timestamp"] = pd.Series(etf_df["Timestamp"]).apply(lambda x: format_date_pd(x))

print(etf_df)
etf_df.set_index(etf_df["Timestamp"], inplace=True)
etf_df.drop(columns=['Timestamp'], inplace=True)
etf_df = etf_df[datetime(2018,1,1):]
print(etf_df)